# 特征选择与稀疏学习

## 过滤式选择

&emsp; 过滤式方法先对数据集进行特征选择，然后再训练学习器，特征选择过程与后续学习器无关。相当于先用特征选择过程对初始特征进行过滤，再用过滤后的特征训练模型。

&emsp; Relief式一种著名的特征选择方法，该方法设计了一个相关统计量来度量特征的重要性，该统计量是一个向量，其每个分类分别对应于一个初始特征，而特征子集的重要性则是由子集中的每个特征所对应的相关统计量之和来决定，于是，最终只需要指定一个阈值$\tau$，然后选择比$\tau$大的相关统计量分量所对应的特征即可；也可指定欲选区的特征个数$k$，然后选择相关统计量分量最大的$k$个特征。

&emsp; 给定训练集$\{(x_1,y_1),(x_2,y_2),\cdots,(x_m,y_m)\}$，对每个示例$x_i$，Relief先在$x_i$的同类样本中寻找其最近邻$x_{i,nh}$，称为猜中近邻；再从$x_i$的异类样本中寻找其最近邻$x_{i,nm}$，称为猜错近邻。然后，相关统计量对应于属性$j$的分量为

$$\delta^j=\sum_i-diff(x_i^j,x_{i,nh}^j)^2+diff(x_i^j,x_{i,nm}^j)^2$$

&emsp; $diff$取决于属性$j$的类型：若为离散型，则$x_a^j=x_b^j$时，$diff(x_a^j,x_b^j)=0$，否则为1；若为连续型，$diff(x_a^j,x_b^j)=|x_a^j,x_b^j|$，这里$x_a^j,x_b^j$已规范化到$[0,1]$

## 包裹式选择

&emsp; 包裹式特征选择直接把最终将要使用的学习器的性能作为特征子集的评价准则。LVW是一个典型的包裹式特征选择方法，使用随机策略来进行子集搜索，并以最终分类器的误差作为特征子集的评价准则。

* 首先进行初始化$E=\infty,\ d=|A|,\ A^{*}=A$

* 循环直到$t=T$：随机产生特征子集$A',\ d'=|A'|, E'=CrossValidation(L(A'))$，若$E'<E$或$E'=E,d'<d$，则$t=0,\ E=E',\ d=d',\ A^{*}=A'$；否则$t=t+1$

&emsp; LVW算法中特征子集搜索采用了随机策略，每次特征子集评价都需训练学习器，计算开销很大。且若初始特征较多，算法可能长时间无法达到停止条件。

## 嵌入选择与$L_1$正则化

&emsp; 嵌入式特征选择是将特征选择过程与学习器训练过程融为一体，两者在同一个优化过程中完成。

&emsp; 给定数据集$D=\{(x_1,y_1),(x_2,y_2),\cdots,(x_m,y_m)\}$，其中$x\in R^d,y\in R$。考虑简单的线性回归模型，以平方误差为损失函数，则优化目标为

$$min_w\sum_{i=1}^m(y_i-w^Tx_i)^2$$

&emsp; 当样本特征很多，而样本数相对较少时，上式很容易陷入过拟合，为了缓解过拟合问题，可引入正则化项，使用$L_2$范数正则化，称为岭回归(ridge regression)，则有

$$min_w\sum_{i=1}^m(y_i-w^Tx_i)^2+\lambda||w||_2^2$$

&emsp; 若使用$L_1$范数，称为LASSO，则有

$$min_w\sum_{i=1}^m(y_i-w^Tx_i)^2+\lambda||w||_1$$

&emsp; $L_1$范数和$L_2$范数都有助于降低过拟合风险，但前者还会带来额外的好处：它比后者更易获得稀疏(sparse)解，即它求得的$w$会有更少色非零分量。

&emsp; $L_1$正则化问题的求解可使用近端梯度下降(PGD)，令$\Delta$表示微分算子，对优化目标

$$min_xf(x)+\lambda||x||_1$$

&emsp; 若$f(x)$可导，且满足存在常数$L>0$使得

$$||\Delta f(x')-\Delta f(x)||_2^2\leq L||x'-x||_2^2, \forall x, x'$$

&emsp; 则在$x_k$附近可将$f(x)$通过二阶泰勒展开式近似为

$$\hat{f}(x)\approx \frac{L}{2}||x-(x_k-\frac{1}{L}\Delta f(x_k))||_2^2+const$$

&emsp; 令$z=x_k-\frac{1}{L}\Delta f(x_k)$，求解

$$x_{k+1}=argmin_x\frac{L}{2}||x-z||_2^2+\lambda||x||_1$$

&emsp; 上式有解(分段二次函数最小值)

$$x_{k+1}^i=\begin{cases} 
		z^i-\frac{\lambda}{L} & \frac{\lambda}{L}<z^i\\
		0 & |z^i|\leq \frac{\lambda}{L}\\
      z^i-\frac{\lambda}{L} & z^i<-\frac{\lambda}{L}
	\end{cases}$$

## 稀疏表示与字典学习

&emsp; 不妨把数据集$D$考虑成一个矩阵，其每行对应于一个样本，每列对应于一个特征。特征选择所考虑的问题时特征具有“稀疏性”，即矩阵中的许多列与当前学习任务无关，通过特征选择去除这些列，则学习器训练过程仅需要在较小的矩阵上进行，学习任务的难度降低，涉及的计算和存储开销会减小，学得模型的可解释性也会提高。

&emsp; 在很多场景下，矩阵每一行都有大量的零元素，对不同的行，零元素出现的列往往很不相同。当样本具有很多这样的系数表达形式时，对学习任务来说会有不少好处；同时稀疏矩阵已经具有很多高效的储存方法。

&emsp; 在一般的学习任务中，将普通稠密表达的样本转化为合适的稀疏表示形式，从而使学习任务得以简化，模型复杂度得以降低，通常称为“字典学习”，亦称“稀疏编码”。给定数据集$\{x_1,x_2,\cdots,x_m\}$，字典学习的最简单形式为

$$min_{B,\alpha_i}\sum_{i=1}^m||x_i-B\alpha_i||_2^2+\lambda\sum_{i=1}^m||\alpha_i||_1$$

&emsp; 其中$B\in R^{d\times k}$为字典矩阵，$k$称为字典的词汇量，通常由用户指定，$\alpha_i\in R^k$则是样本$x_i\in R^d$的稀疏表示。我们可采用交替优化的策略来进行求解。

* 第一步固定字典$B$，采用LASSO的方法进行求解

$$min_{\alpha_i}||x_i-B\alpha_i||_2^2+\lambda||\alpha_i||_1$$

* 第二部，以初值$\alpha_i$来更新字典$B$

$$min_B||X-BA||_F^2$$

&emsp; 其中$X=(x_1,x_2,\cdots,x_min R^{d\times m},\ A=(\alpha_1, \alpha_2,\cdots,\alpha_m)\in R^{k\times m},\ ||\cdot||_F$是矩阵中的Frobenius范数，这里采用基于逐列更新策略的KSVD。令$b_i$表示字典矩阵$B$的第$i$列，$\alpha^i$表示稀疏矩阵$A$的第$i$行

$$min_B||X-BA||_F^2=min_{b_i}||X-\sum_{j=1}^kb_j\alpha^j||_F^2=min_{b_i}||(X-\sum_{j\neq i}b_j\alpha^j)-b_i\alpha^i||_F^2=min_{b_i}||E_i-b_i\alpha^i||_F^2$$

&emsp; KSVD对$E_i$与$\alpha_i$进行专门处理：$\alpha_i$仅保留非零元素，$E_i$则仅保留$b_i$与$\alpha_i$的非零元素的乘积项，然后再进行奇异值分解，这样就保持了第一部所得到的稀疏性。